In [1]:
#from mpl_toolkits import mplot3d
import pandas as pd
from numpy import array
import numpy as np
import h5py
import mplcursors
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import pearsonr
%matplotlib inline
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "pgf.rcfonts": False,

    "text.usetex": True,
    "font.family": "serif",
    "font.sans-serif": "computer-modern-roman",

    "figure.dpi": 300,
    "savefig.dpi": 300,
})


CMAPS = plt.colormaps
CMAP = CMAPS['hsv']

h = 0.6774  # co-moving constant


In [3]:
g_dat = pd.read_pickle('../beta_data.pickle')

In [5]:
h5dat = h5py.File('../main_branch_orbit_data_snapshot_1200.hdf5')

flydat = pd.DataFrame((list(x) for x in h5dat.values() if x.shape != ()), index=(
    k for k in h5dat.keys() if h5dat[k].shape != ())).transpose().convert_dtypes()


In [6]:
dat = g_dat.merge(flydat, left_index=True, right_index=True)



# remove bad data
dat = dat[dat['d_innermost'] < 20000].copy()

# M_tot calculation
dat['M_tot'] = (
    dat['stars_mass_within_r50_stars'] + 
    dat['gas_mass_within_r50_stars'] + 
    dat['dm_mass_within_r50_stars']
)

s_filt = dat['is_satellite']

# The strange galaxies
The following galaxies seem to be satellites that have not yet reached their minimum orbit distance

In [8]:
is_weird_galaxy = dat['is_satellite'] & (dat['d_innermost'] == 0)

In [23]:
print(*(dat.loc[is_weird_galaxy, 'galaxyID']), sep=', ') # galaxy ids

92.0, 162.0, 178.0, 348.0, 359.0, 414.0, 495.0, 565.0, 615.0, 682.0, 748.0, 870.0, 904.0, 941.0, 1162.0, 1269.0, 1273.0, 1345.0, 1348.0, 1492.0, 1906.0, 2239.0, 2327.0, 2361.0, 2629.0, 2739.0, 2768.0, 3243.0, 3452.0, 3454.0, 3468.0, 3772.0, 3919.0, 3928.0, 4036.0, 4126.0, 4513.0, 4549.0, 4734.0, 4739.0, 4787.0, 4899.0, 5478.0, 5665.0, 5818.0, 5912.0, 6278.0, 6466.0, 6495.0, 6635.0, 6815.0, 7254.0, 7689.0, 8353.0, 8544.0, 8546.0, 8705.0, 9224.0, 9973.0, 10640.0, 11010.0, 12208.0, 12682.0, 12738.0, 13299.0, 13514.0, 13781.0, 14240.0, 14442.0, 15840.0, 17588.0, 17795.0, 18379.0, 19129.0, 20747.0, 22246.0, 23231.0, 25513.0, 28823.0, 29309.0, 30409.0, 31281.0, 49407.0


In [13]:
sum(is_weird_galaxy) # how many are there?

83

In [14]:
sum(is_weird_galaxy) / sum(dat['is_satellite']) # fraction of satellites with this property

0.18526785714285715

In [22]:
dat[dat['is_satellite'] & (dat['d_innermost'] != 0)]['M_\\star/M_tot'].mean() # mean stellar mass to dm mass (within r_50), for satellites without this property

0.1872992914077583

In [18]:
dat[is_weird_galaxy]['M_\\star/M_tot'].mean() # same calculation, for galaxies with this property

0.12404652083412257